### Обработать данные из датасета по e-commerce, для дальнейшей работы с ним

In [38]:
import pandas as pd
import datetime as dt

In [39]:
df = pd.read_csv(r'C:\Users\Артем\New_Jupyter\data\sravni_ru_data.csv', encoding='Windows-1251')

## Смотрим информацию по датасету

In [40]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [42]:
# смотрим где и сколько null
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [43]:
# смотри null в процентах
for col in df.columns:
  print(col, round(df[col].isna().sum() / df.shape[0] *100,2))

InvoiceNo 0.0
StockCode 0.0
Description 0.27
Quantity 0.0
InvoiceDate 0.0
UnitPrice 0.0
CustomerID 24.93
Country 0.0


In [68]:
# смотрим сколько раз встречаются товары с отрицательным количеством продаж (возможно это возвраты, но мы этого не знаем)
df.Quantity[df.Quantity < 0].count()

10624

In [48]:
# смотрим сколько раз встречаются товары с отрицательной ценой
df.UnitPrice[df.UnitPrice < 0].count()

2

In [49]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [50]:
# смотрим уникальные страны и кол-во повторов по датасету
df['Country'].value_counts()

United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

In [51]:
# смотрим топ уникальных товаров и кол-во по датасету
df['Description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
                                      ... 
PINK FEATHER CHRISTMAS DECORATION        1
SET 10 CARDS SNOWY ROBIN 17099           1
FUNKY MONKEY MUG                         1
mix up with c                            1
wrongly coded 23343                      1
Name: Description, Length: 4223, dtype: int64

## Работа с данными

In [90]:
# перезаписываем датасет, что бы к оригиналу можно было быстро обратиться
df_1 = df

In [91]:
# удаляем строчки где цена или количесвто меньше 0
df_1 = df_1[df_1.UnitPrice > 0]
df_1 = df_1[df_1.Quantity > 0]

In [92]:
df_1['Invoice_dt'] = pd.to_datetime(df_1["InvoiceDate"])

In [93]:
# смотрим мин и макс даты
print(df_1['Invoice_dt'].min())
print(df_1['Invoice_dt'].max())

2010-12-01 08:26:00
2011-12-09 12:50:00


In [94]:
# принудительно меняем null в CustomerID на 99999
df_1 = df_1.fillna({"CustomerID":99999})
df_1.CustomerID.isna().sum()

0

In [101]:
# добавляем фичи для анализа (месяц, квартал, номер недели, номер часа в сутках, стоимость сделки)
df_1['Invoice_dt'] = df_1['Invoice_dt'].dt.strftime('%Y-%m-%d')
df_1['Invoice_dt'] = pd.to_datetime(df_1["Invoice_dt"])
df_1['year'] = df_1['Invoice_dt'].dt.strftime('%Y')
df_1['month'] = df_1['Invoice_dt'].dt.strftime('%m')
df_1['y_m'] = df_1['Invoice_dt'].dt.strftime('%Y-%m')
df_1['quarter'] = df_1['Invoice_dt'].dt.to_period('Q')
df_1['weekday_num'] = df_1['Invoice_dt'].dt.strftime('%A')
df_1['hours'] = pd.to_datetime(df_1["InvoiceDate"]).dt.strftime('%H')
df_1['revenue'] = df_1.UnitPrice * df_1.Quantity

In [102]:
df_1['revenue'].sum()

10666684.544

In [65]:
# считаем 25 и 75 квантили по выручке и по кол-ву заказов для распределения по группам
revenue_1 = df_1.groupby('CustomerID')['revenue'].sum()
revenue_1 = pd.DataFrame(revenue_1).reset_index()
rev_25, rev_75 = revenue_1['revenue'].quantile([0.25,0.75])

order_1 = df_1.drop_duplicates(subset=['InvoiceNo']).groupby('CustomerID')['InvoiceNo'].count()
order_1 = pd.DataFrame(order_1).reset_index()
ord_25, ord_75 = order_1['InvoiceNo'].quantile([0.25,0.75])

df_cust = df_1.groupby(['CustomerID'], as_index=False)\
                .agg({'revenue':'sum', 'InvoiceNo':'nunique'})\
                .rename(columns = {'InvoiceNo':'orders'})

df_cust['rev_25'] = rev_25
df_cust['rev_75'] = rev_75
df_cust['ord_25'] = ord_25
df_cust['ord_75'] = ord_75

# итоговая таблица для анализа по клиентам
df_cust.head()

,CustomerID,revenue,orders,rev_25,rev_75,ord_25,ord_75
0,12346.0,77183.60,1,307.43,1662.06,1.0,5.0
1,12347.0,4310.00,7,307.43,1662.06,1.0,5.0
2,12348.0,1797.24,4,307.43,1662.06,1.0,5.0
3,12349.0,1757.55,1,307.43,1662.06,1.0,5.0
4,12350.0,334.40,1,307.43,1662.06,1.0,5.0


In [63]:
df_tbl = df_1.groupby(['Country', 'CustomerID', 'Invoice_dt', 'month', 'y_m', 'quarter', 'weekday_num', 'hours'], as_index=False)\
    .agg({'revenue':'sum', 'Quantity':'sum', 'InvoiceNo':'nunique', 'StockCode':'count'})\
    .rename(columns = {'InvoiceNo':'orders', 'Quantity':'qty', 'StockCode':'producs'})

# итоговая таблица для общего мониторинга
df_tbl.head()

,Country,CustomerID,Invoice_dt,month,y_m,quarter,weekday_num,hours,revenue,qty,orders,producs
0,Australia,12386.0,2010-12-08,12,2010-12,2010Q4,Wednesday,09,258.90,214,1,8
1,Australia,12386.0,2011-01-06,01,2011-01,2011Q1,Thursday,12,143.00,140,1,2
2,Australia,12388.0,2011-01-17,01,2011-01,2011Q1,Monday,11,431.30,204,1,19
3,Australia,12388.0,2011-02-07,02,2011-02,2011Q1,Monday,13,124.05,37,1,6
4,Australia,12388.0,2011-03-09,03,2011-03,2011Q1,Wednesday,15,210.90,90,1,10


In [66]:
# Создаем файл
writer = pd.ExcelWriter('df_on_sheets.xlsx', engine='xlsxwriter')
# Записываем на листы
df_tbl.to_excel(writer, sheet_name='df_tbl', index=False)
df_cust.to_excel(writer, sheet_name='df_cust', index=False)
# Сохраняем и закрвыаем 
writer.save()